## Denial Analysis
<p>For this project I decided to go through only one single claim. There is a process that you can follow where there is a defined starting point and a defined stopping point. To prevent a denial, you need to ensure that preauthorization has been done. Insurance verification would have been done, and the actual procedure has already happened. Now, all those steps leading up to the actual the actual visit or the actual procedure, there are so many little key things that need to be done within that process to make sure that a claim doesn't get denied, for example, within the insurance verification. If the billing system spells the tame of the patient differently than how the payer has it, then that will eventually be denied. For example, I legally changed my name by removing the dash from Shaun-jay to shaunjay and this has been an issue a few times and as cause the claims ended to be denied. I guess years the first name was issued but the EMR system seem to not totally update this information. A single letter within the name can actually cause for a denial and then a complete rework of submitting </p>

#### General Terminolgy
* Billing Provider: This is the provider the claim is sent under
* Rendering/Servicing Provider
    - Someone else can actually be the service provider thats bill under their name
        - Often Nurse practitioner - They are doing the service but its bill by another provider
* Referring Provider: This could be the primary care doctor that sent the patient to a specialty provider
* Service Date: The date the actual service was rendered
* Charge Post Date: Once the physician has finised their documentation encoding, they will submit the claim, this is the date at which the charge was close within billing system. This Date cannot change
* Transaction Post Date: The date that every transaction is recorded. The charge Post Date and transaction Post Date date will be the same
    - So you'll actually see a date for every transaction that happens. So the date that you send the invoice to payer, the date that the payer actually pays, or the date the payer actually denies, the fee claim, that would also be your denial rate.
    - Your transactions could also be the date that the organization decided decided to write something off and actually have a date for every transaction.
* Denial Date: The date that your claim is deny

<p>It's important to note that those dates are different, often doing different analyses.
Those dates are important to remember because they can impact how you are doing data.</p>

In [404]:
import pandas as pd
import numpy as np
%store -r denial

In [18]:
denial.head(4)

,State,Patient Number,Payer Name,(Non)Facility,Medicare POS Number,Medicare POS Desc,Location Name,Cpt Code,CPT Modifier,CPT Desc,...,RARC,CARC,Denial Category,Charges,Denial Amount,Payments,Adjustments,Net Ar,Denial Count,Units
0,Utah,80894,Medicare,F,22,On Campus-Outpatient Hospital,Northridge Medical Center,77067,26,Scr mammo bi incl cad,...,NaN,NaN,,0.0,0.0,-38.184324,0.000000,-38.184324,0,0
1,Utah,80894,Medicare,F,22,On Campus-Outpatient Hospital,Northridge Medical Center,77067,26,Scr mammo bi incl cad,...,NaN,NaN,,0.0,0.0,0.000000,-53.976784,-53.976784,0,0
2,Utah,80894,Medicare,F,22,On Campus-Outpatient Hospital,Northridge Medical Center,77067,26,Scr mammo bi incl cad,...,NaN,NaN,,0.0,0.0,0.000000,0.720459,0.720459,0,0
3,Utah,80894,Medicare,F,22,On Campus-Outpatient Hospital,Northridge Medical Center,77067,26,Scr mammo bi incl cad,...,NaN,NaN,,0.0,0.0,0.000000,-0.778096,-0.778096,0,0


In [17]:
denial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132758 entries, 0 to 132757
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   State                132758 non-null  object 
 1   Patient Number       132758 non-null  int64  
 2   Payer Name           132758 non-null  object 
 3   (Non)Facility        132758 non-null  object 
 4   Medicare POS Number  132758 non-null  int64  
 5   Medicare POS Desc    132758 non-null  object 
 6   Location Name        132758 non-null  object 
 7   Cpt Code             132758 non-null  object 
 8   CPT Modifier         34697 non-null   object 
 9   CPT Desc             132758 non-null  object 
 10  Transaction          132758 non-null  object 
 11  RARC                 1863 non-null    object 
 12  CARC                 2263 non-null    object 
 13  Denial Category      132758 non-null  object 
 14  Charges              132758 non-null  float64
 15  Denial Amount    

## Denial Details

In [62]:
rpt = denial[(denial['Patient Number']== 93111)&(denial['Cpt Code'].str.contains('99392'))&(denial['Payer Name'].str.contains('Commercial'))&(denial['Medicare POS Desc'].str.contains('Office'))]
rpt = rpt.groupby(["Transaction"]).apply(
    lambda d: pd.Series(
        {
            "Sum of Charges": d["Charges"].sum(),
            "Sum of Denail Amount":d["Denial Amount"].sum(),
            "Sum of Payments":d["Payments"].sum(),
            "Sum of Adjustments":d["Adjustments"].sum(),
            "Sum of Net Ar": d["Net Ar"].sum()
        }
    )

rpt.loc['Total'] = pd.Series(rpt[["Sum of Denail Amount","Sum of Payments","Sum of Adjustments","Sum of Net Ar"]]
                  .sum(), index = ["Sum of Denail Amount","Sum of Payments","Sum of Adjustments","Sum of Net Ar"])
rpt.style.format({   
                    "Sum of Charges": "${:20,.2f}", 
                    "Sum of Denail Amount": "{:20,.0f}",
                    "Sum of Payments": "${:20,.2f}",
                    "Sum of Adjustments":"${:20,.2f}",
                     "Sum of Net Ar":"${:20,.2f}",
                 }).format_index(str.upper, axis=1).background_gradient(cmap='Reds')

,SUM OF CHARGES,SUM OF DENAIL AMOUNT,SUM OF PAYMENTS,SUM OF ADJUSTMENTS,SUM OF NET AR
Transaction,,,,,
bill,$ 0.00,0,$ 0.00,$ 0.00,$ 0.00
contractual/differential w/o,$ 0.00,0,$ 0.00,$ -43.75,$ -43.75
denial,$ 0.00,198,$ 0.00,$ 0.00,$ 0.00
invoice,$ 0.00,0,$ 0.00,$ 0.00,$ 0.00
payment insurance,$ 0.00,0,$ -143.35,$ 0.00,$ -143.35
"pr preventive visit,est,age 1-4",$ 197.89,0,$ 0.00,$ 0.00,$ 197.89
withholding w/o,$ 0.00,0,$ 0.00,$ -10.79,$ -10.79
Total,$ nan,198,$ -143.35,$ -54.53,$ 0.00


## Denial RARC

In [108]:
rpt1 = denial[(denial['Patient Number']== 93111)&(denial['Cpt Code'].str.contains('99392'))&(denial['Payer Name'].str.contains('Commercial'))&(denial['Medicare POS Desc'].str.contains('Office'))]
rpt1 = rpt1[['RARC','CARC']].dropna()
rpt1.style.set_properties(**{"font-size":".9rem"}) 

,RARC,CARC
127562,Additional information has been requested from the member. The charges will be reconsidered upon receipt of that information.,Information requested from the patient/insured/responsible party was not provided or was insufficient/incomplete. At least one Remark Code must be provided.


## Denial Category

In [201]:
rpt2 = denial[denial['Denial Category']!=' '].groupby(["Denial Category"]).apply(
    lambda d: pd.Series(
        {
            "Count of Patient Number": d["Patient Number"].count(),
            "Sum of Denail Amount":d["Denial Amount"].sum(),
            "diff":(d["Denial Amount"].sum()/ d["Patient Number"].count()) #* 100000
        }
    )).sort_values(by="Count of Patient Number", ascending=False)

rpt2.loc['Grand Total'] = pd.Series(rpt2[["Count of Patient Number","Sum of Denail Amount","diff"]]
                  .sum(), index = ["Count of Patient Number","Sum of Denail Amount"])
rpt2.style.format({   
                    "Count of Patient Number": "{:20,.0f}", 
                    "Sum of Denail Amount": "${:20,.2f}",
                    "diff": "{:20,.2f}"
                 }).format_index(str.upper, axis=1,).background_gradient(cmap='Reds')


,COUNT OF PATIENT NUMBER,SUM OF DENAIL AMOUNT,DIFF
Denial Category,,,
Information Required,693,"$ 85,832.97",123.86
Benefit Coverage,569,"$ 59,093.78",103.86
Contractual,226,"$ 29,223.91",129.31
Bundled,208,"$ 14,441.30",69.43
Reimbursement Adj.,146,"$ 8,966.74",61.42
Physician Enrollment,111,"$ 21,139.96",190.45
Miscellaneous,98,"$ 12,025.86",122.71
Medical Necessity,63,"$ 9,251.65",146.85
Chargemaster,32,"$ 1,875.11",58.60


In [168]:
denial[['Denial Category']].notnull().groupby(["Denial Category"]).count()

""
Denial Category
True


In [223]:
rpt3 = denial[denial['Denial Category'].str.contains('Physician Enrollment')].groupby(["Location Name"]).apply(
    lambda d: pd.Series(
        {
            "Count of Patient Number": d["Patient Number"].count(),
            "Sum of Denail Amount":d["Denial Amount"].sum(),
            "diff":(d["Denial Amount"].sum()/ d["Patient Number"].count()) #* 100000
        }
    )).sort_values(by="Count of Patient Number", ascending=False)

rpt3.loc['Grand Total'] = pd.Series(rpt3[["Count of Patient Number","Sum of Denail Amount","diff"]]
                  .sum(), index = ["Count of Patient Number","Sum of Denail Amount","diff"])
rpt3.style.format({   
                    "Count of Patient Number": "{:20,.0f}", 
                    "Sum of Denail Amount": "{:20,.2f}",
                    "diff": "{:20,.2f}"
                 }).format_index(str.upper, axis=1,).background_gradient(cmap='Reds')

,COUNT OF PATIENT NUMBER,SUM OF DENAIL AMOUNT,DIFF
Location Name,,,
Paradise Medical Center,40,"4,711.80",117.80
Yellowstone Hospital,35,"7,630.87",218.02
Northridge Medical Center,25,"8,301.61",332.06
Pineview Hospital,4,59.56,14.89
Goldriver Medical Center,3,165.23,55.08
Green Valley Hospital,3,235.35,78.45
Lakeview Hospital,1,35.54,35.54
Grand Total,111,"21,139.96",851.84


## Denial exact issues

In [417]:
rpt4 = denial[(denial['Denial Category'].str.contains('Physician Enrollment'))
              &(denial['Location Name'].str.contains('Northridge Medical Center')) &(denial['Cpt Code'].str.contains('93306'))]
rpt4 = rpt4[['Cpt Code','CPT Desc','CARC']] .drop_duplicates()
rpt4.style.set_properties(**{"font-size":".9rem"})

,Cpt Code,CPT Desc,CARC
912,93306,Tte w/doppler complete,"This provider was not certified/eligible to be paid for this procedure/service on this date of service. Usage: Refer to the 835 Healthcare Policy Identification Segment (loop 2110 Service Payment Information REF), if present."


In [435]:
rpt5 = denial[(denial['Denial Category'].str.contains('Physician Enrollment'))&(denial['Location Name'].str.contains('Northridge Medical Center')) 
              &(denial['Cpt Code'].str.contains('93306'))].groupby(['Cpt Code','CPT Desc', 'CARC']).apply(
    lambda d: pd.Series(
        {
            'Cpt Code': d['Cpt Code'].to_string(),
            'CPT Desc': d['CPT Desc'].to_string(),
            'CARC': d['CARC'].to_string(),
            "Count of Patient Number": d["Patient Number"].count(),
            "Sum of Denail Amount":d["Denial Amount"].sum(),
            "diff":(d["Denial Amount"].sum()/ d["Patient Number"].count()) #* 100000
        }
    )).sort_values(by="Count of Patient Number", ascending=False)

rpt5.loc['Grand Total'] = pd.Series(rpt5[["Count of Patient Number","Sum of Denail Amount","diff"]]
                  .sum(), index = ['Cpt Code','CPT Desc', 'CARC',"Count of Patient Number","Sum of Denail Amount","diff"])
rpt5.style.format({   
                   'Cpt Code': lambda x:x.lower(),
                   'CPT Desc': lambda x:x.lower(),
                   'CARC': lambda x:x.lower(),
                    "Count of Patient Number": "{:20,.0f}", 
                    "Sum of Denail Amount": "{:20,.2f}",
                    "diff": "{:20,.2f}"
                 }).format_index(str.upper, axis=1,).background_gradient(cmap='Reds')

AttributeError: 'float' object has no attribute 'lower'

In [412]:
rpt6 = denial.groupby(["Patient Number"]).apply(
    lambda d: pd.Series(
        {
            "Sum of Denial Count": np.count_nonzero(d["Denial Amount"]),
            "Sum of Charges":d["Charges"].sum(),
            "Sum of Payments":d["Payments"].sum() * -1
        }
    )).sort_values(by="Sum of Charges", ascending=False).head(20)

rpt6.loc['Grand Total'] = pd.Series(rpt6[["Sum of Denial Count", "Sum of Charges", "Sum of Payments"]]
                  .sum(), index = ["Sum of Denial Count", "Sum of Charges", "Sum of Payments"])
rpt6.style.format({   
                    "Sum of Denial Count": "{:20,.0f}", 
                    "Sum of Charges": "{:20,.2f}",
                    "Sum of Payments": "{:20,.2f}"
                 }).format_index(str.upper, axis=1,).background_gradient(cmap='Reds')

,SUM OF DENIAL COUNT,SUM OF CHARGES,SUM OF PAYMENTS
Patient Number,,,
71777,1,"24,855.83","12,206.75"
46461,4,"24,123.85","6,589.26"
41363,1,"10,624.37","5,223.78"
58487,5,"9,063.37","1,311.63"
58794,1,"8,734.84",674.67
75861,0,"8,646.47","4,494.61"
22095,1,"8,243.97","2,726.27"
45311,0,"8,042.24","4,193.11"
52760,0,"8,042.24","4,507.15"


In [373]:
denial.groupby('Patient Number')['Payer Name'].value_counts().to_frame()

,,Payer Name
Patient Number,Payer Name,
10010,Medicare,6
10019,Medicare,6
10029,Medicare,5
10061,Commercial,18
10066,Commercial,14
...,...,...
99946,Commercial,5
99960,Medicare,12
99964,Medicare,6


In [318]:
denial[(denial['Patient Number']== 62286)&(denial['Denial Category']==' ')].head(10)

,State,Patient Number,Payer Name,(Non)Facility,Medicare POS Number,Medicare POS Desc,Location Name,Cpt Code,CPT Modifier,CPT Desc,...,RARC,CARC,Denial Category,Charges,Denial Amount,Payments,Adjustments,Net Ar,Denial Count,Units
71249,Utah,62286,Medicaid,F,21,Inpatient hospital,Yellowstone Hospital,59400,NaN,Obstetrical care,...,NaN,NaN,,0.000000,0.0,-1937.121973,0.000000,-1937.121973,0,0
71250,Utah,62286,Medicaid,F,21,Inpatient hospital,Yellowstone Hospital,59400,NaN,Obstetrical care,...,NaN,NaN,,0.000000,0.0,0.000000,-2020.599149,-2020.599149,0,0
71251,Utah,62286,Medicaid,F,21,Inpatient hospital,Yellowstone Hospital,59400,NaN,Obstetrical care,...,NaN,NaN,,3957.721121,0.0,0.000000,0.000000,3957.721121,0,1
71252,Utah,62286,Medicaid,F,21,Inpatient hospital,Yellowstone Hospital,59400,NaN,Obstetrical care,...,NaN,NaN,,0.000000,0.0,0.000000,0.000000,0.000000,0,0
